# Affine Cipher
1. Given plaintext (or ciphertext) and key (a, b), encrypt and decrpt
2. Given ciphertext, use frequency analysis to guess key, then decrypt - may not always work!
3. Given plaintext and ciphertext, find key

In [ ]:
# Given plaintext (or ciphertext) and key (a, b), encrypt and decrpt

def mod_inverse(a, m):
    # Extended Euclidean Algorithm
    t, new_t = 0, 1
    r, new_r = m, a
    while new_r != 0:
        q = r // new_r
        t, new_t = new_t, t - q * new_t
        r, new_r = new_r, r - q * new_r
    if r > 1:
        raise ValueError(f"No inverse for {a} mod {m}")
    if t < 0:
        t += m
    return t

def affine_encrypt(plaintext, a, b, m=26):
    ciphertext = ""
    for ch in plaintext:
        if ch.isalpha():
            x = ord(ch.lower()) - ord('a')
            y = (a * x + b) % m
            ciphertext += chr(y + ord('a'))
        else:
            ciphertext += ch
    return ciphertext

def affine_decrypt(ciphertext, a, b, m=26):
    plaintext = ""
    a_inv = mod_inverse(a, m)
    for ch in ciphertext:
        if ch.isalpha():
            y = ord(ch.lower()) - ord('a')
            x = (a_inv * (y - b)) % m
            plaintext += chr(x + ord('a'))
        else:
            plaintext += ch
    return plaintext

# Example:
msg = "hello"
a, b = 5, 8  # a must be coprime with 26
enc = affine_encrypt(msg, a, b)
dec = affine_decrypt(enc, a, b)
print("Encrypted:", enc)  # rclla
print("Decrypted:", dec)  # hello

# works the same for capitals

In [ ]:
# Given ciphertext, use frequency analysis to guess key, then decrypt - may not always work!

from collections import Counter

def mod_inverse(a, m):
    t, new_t = 0, 1
    r, new_r = m, a
    while new_r != 0:
        q = r // new_r
        t, new_t = new_t, t - q * new_t
        r, new_r = new_r, r - q * new_r
    if r > 1:
        raise ValueError(f"No inverse for {a} mod {m}")
    return t % m

def affine_decrypt(ciphertext, a, b, m=26):
    a_inv = mod_inverse(a, m)
    plaintext = ""
    for ch in ciphertext:
        if ch.isalpha():
            y = ord(ch.lower()) - ord('a')
            x = (a_inv * (y - b)) % m
            plaintext += chr(x + ord('a'))
        else:
            plaintext += ch
    return plaintext

def affine_frequency_attack(ciphertext):
    text_only = [ch for ch in ciphertext.lower() if ch.isalpha()]
    counts = Counter(text_only).most_common(2)
    y1, y2 = ord(counts[0][0]) - ord('a'), ord(counts[1][0]) - ord('a')
    x1, x2 = 4, 19  # e, t

    # Solve for a
    delta_x = (x1 - x2) % 26
    delta_y = (y1 - y2) % 26
    a = (delta_y * mod_inverse(delta_x, 26)) % 26
    b = (y1 - a*x1) % 26

    plaintext = affine_decrypt(ciphertext, a, b)
    return plaintext, a, b

# Example
ciphertext = "attack"
plaintext, a, b = affine_frequency_attack(ciphertext)
print("Guessed keys: a =", a, ", b =", b)
print("Decrypted text:", plaintext)


In [ ]:
# Given plaintext and ciphertext, find key

# Helper: modular inverse
def mod_inverse(a, m=26):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None  # No inverse if gcd(a, m) ≠ 1

def affine_key(p1, c1, p2, c2, m=26):
    """
    Recover Affine cipher keys (a, b) from two plaintext-ciphertext letter pairs.
    Returns (a, b)
    """
    P1 = ord(p1.lower()) - ord('a')
    C1 = ord(c1.lower()) - ord('a')
    P2 = ord(p2.lower()) - ord('a')
    C2 = ord(c2.lower()) - ord('a')

    # Solve for a
    diffP = (P1 - P2) % m
    diffC = (C1 - C2) % m
    inv = mod_inverse(diffP, m)
    if inv is None:
        raise ValueError("No modular inverse exists; invalid pairs.")
    a = (diffC * inv) % m

    # Solve for b
    b = (C1 - a * P1) % m
    return a, b

# Example
print("\nAffine example:")
# in format for affine_key(p1, c1, p2, c2) where p is for plaintext letter and c is for ciphertext letter
a, b = affine_key("H", "R", "E", "C") 
print("a =", a, ", b =", b)
